In [1]:
import os
from typing import Any
import pandas as pd
from waffleml.modules.experiments import VertexExperiment
from waffleml.results_analysis import retrieve_inputs_outputs, get_pipeline_runs, get_run_metadata, \
    retrieve_artifacts, retrieve_final_model, retrieve_hyper_parameters, get_hyperparameters, get_metrics
from waffleml.modules.storage import GoogleCloudStorage as storage
import numpy as np
import pandas as pd

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/witj/.google/service_accounts/waffleml_dev_key.json"

In [3]:
experiment = VertexExperiment("common-codebase")

In [4]:
MODELS = ["tomorrow_is_today", "lgbm_baseline", "xgb_ar"]

In [11]:
fields={"model": "xgb_baseline", "phase": {"name": "hyper"}}

hyper_runs = get_pipeline_runs(experiment, fields)

In [12]:
hyper_runs

[]

In [ ]:
def hyper_leaderboard_score(model_name, experiment):
    fields={"model": model_name, "phase": {"name": "hyper"}}
    hyper_runs = get_pipeline_runs(experiment, fields)
    scores = []
    for run in hyper_runs:
        run_metadata = get_run_metadata(run.display_name)
        run_artifacts = retrieve_inputs_outputs(run)
        metric = run_artifacts["outputs"]["metrics"]["mae"]
        print(f"{model_name} / hyper fold:{run_metadata['hyper']} -> mae: {metric}")
        scores.append(metric)
    print(f"{model_name} / average mae / {np.mean(scores)}")
    return np.mean(scores)

In [ ]:
leaderboard = pd.DataFrame({ "model" : MODELS, "metric" : [hyper_leaderboard_score(mn, experiment) for mn in MODELS]})

In [ ]:
leaderboard.sort_values(by="metric", ascending=True)